In [1]:
def jobs_manager():
    from IPython.lib.backgroundjobs import BackgroundJobManager
    from IPython.core.magic import register_line_magic
    from IPython import get_ipython

    jobs = BackgroundJobManager()

    @register_line_magic
    def job(line):
        ip = get_ipython()
        jobs.new(line, ip.user_global_ns)

    return jobs
import sys
import argparse
import random
from nile.api.v1 import (
    clusters,
    Record,
    filters as nf,
    aggregators as na,
    grouping as ng,
    extractors as ne,
    statface as ns
)
from qb2.api.v1 import (
    filters as sf,
    extractors as se,
    resources as sr
)
import nile
from urlparse import urlparse
import json
from itertools import product
from nile.api.v1 import datetime as nd
import re
import pandas as pd
import random
import string
from datetime import datetime

In [2]:
cluster = clusters.YT(proxy="hahn.yt.yandex.net", pool="voice") \
                .env(templates=dict(
                                job_root=('home/voice/talamable/VA-274'),
                                dates='{2018-08-16..2018-08-29}'))

In [14]:
job = cluster.job()
hosts = job.table('//home/voice/talamable/VA-274/commercial_week_hosts')
classify = job.table('home/search-research/hosts-markup')
hosts.join(classify, by='host') \
        .put('$job_root/commercial_week_classification')
job.run()
    

VBox()

In [15]:
from operator import itemgetter
def find_topic(records):
    for rec in records:
        topics = eval(rec['suggestions'])
        topics.pop('unknown', None)
        if len(topics) == 0:
            continue
        else:
            top = sorted(topics.items(), key=itemgetter(1))[-1]
            yield Record(cm=rec['cm'],
                         query=rec['query'],
                         host=rec['host'],
                         clicks=rec['clicks'],
                         topic=top[0],
                         weight=top[1]
                        )
from collections import defaultdict
def determine_query_topic(groups):
    for key, records in groups:
        query_weights_clicked = defaultdict(float)
        query_counts_clicked = defaultdict(int)
        query_weights = defaultdict(float)
        query_counts = defaultdict(int)
        for rec in records:
            if rec['clicks']>0:
                query_weights_clicked[rec['topic']] += rec['weight']*rec['clicks']
                query_counts_clicked[rec['topic']] += rec['clicks']
            query_weights[rec['topic']] += rec['weight']
            query_counts[rec['topic']] += 1
        if query_weights_clicked:
            top_w_c = sorted(query_weights_clicked.items(), key=itemgetter(1))[-1][0]
            top_c_c = sorted(query_counts_clicked.items(), key=itemgetter(1))[-1][0]
        else:
            top_w_c = '-'
            top_c_c = '-'
        yield Record(query=key.query,
                     weights=json.dumps(query_weights),
                     counts=json.dumps(query_counts),
                     click_weights=json.dumps(query_weights_clicked),
                     click_counts=json.dumps(query_counts_clicked),
                     top_w = sorted(query_weights.items(), key=itemgetter(1))[-1][0],
                     top_c = sorted(query_counts.items(), key=itemgetter(1))[-1][0],
                     top_w_c = top_w_c,
                     top_c_c = top_c_c
                    )
job = cluster.job()
classify = job.table('$job_root/commercial_week_classification') \
              .map(find_topic) \
              .groupby('query') \
              .reduce(determine_query_topic) \
              .put('$job_root/commercial_week_queries_with_topics')
job.run()

VBox()

In [16]:
def choose_topic_euristic(records):
    for rec in records:
        if sum(json.loads(rec['click_counts']).values()) >= 2:
            topic = rec['top_w_c']
        else:
            topic = rec['top_w']
        yield Record(query=rec['query'],
                     cat = topic
                    )
job = cluster.job()
classified = job.table('$job_root/commercial_week_queries_with_topics') \
                .map(choose_topic_euristic)
all_q = job.table('$job_root/commercial_week')
all_q.join(classified, by='query', type='left') \
     .put('$job_root/commercial_week_cats')
job.run()

VBox()